In [1]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
from kerastuner import Objective
import tensorflow as tf
import numpy as np



<ipython-input-2-1e3fc80dd604>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Load necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
from kerastuner import Objective



# Split data into train and test sets
#np.random.seed(123)  # For reproducibility
#train_data, test_data = train_test_split(filtered_data_NN, test_size=0.3)

subset_train = pd.read_csv("/content/subset_train.csv")
subset_test = pd.read_csv("/content/subset_test.csv")

# Predictor variables
X_train = subset_train[["VehPower", "VehAge", "DrivAge", "BonusMalus",
                      "VehBrand_encoded", "VehGas_encoded",
                      "Region_encoded", "Area_encoded", "Density"]]

X_test = subset_test[["VehPower", "VehAge", "DrivAge", "BonusMalus",
                      "VehBrand_encoded", "VehGas_encoded",
                      "Region_encoded", "Area_encoded", "Density"]]

# Target variables
y_train = subset_train["ClaimNb"]
y_test = subset_test["ClaimNb"]

y_train_sev = subset_train["ClaimAmount"]
y_test_sev = subset_test["ClaimAmount"]

# Define the model-building function
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Dense(units=hp.Int('units1', min_value=32, max_value=512, step=32),
                               input_shape=(9,),
                               activation=hp.Choice('activation1', values=['relu', 'tanh', 'sigmoid'])))  # Activation for first layer
        model.add(layers.Dense(units=hp.Int('units2', min_value=16, max_value=256, step=16),
                               activation=hp.Choice('activation2', values=['relu', 'tanh', 'sigmoid'])))
        model.add(layers.Dense(units=hp.Int('units3', min_value=8, max_value=256, step=16),
                               activation=hp.Choice('activation3', values=['relu', 'tanh', 'sigmoid'])))
        model.add(layers.Dense(units=1, activation='linear'))

        model.compile(
            optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='mean_squared_error'
        )
        return model

# Initialize the tuner
tuner1 = RandomSearch(
    MyHyperModel(),
    objective=Objective('val_loss', direction='min'),  # Use 'val_loss' since you're using mean squared error as loss
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir1',
    project_name='helloworld'
)


# Fit the tuner with batch size and epochs
tuner1.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Trial 5 Complete [00h 06m 15s]
val_loss: 0.09864064306020737

Best val_loss So Far: 0.09813291827837627
Total elapsed time: 00h 27m 48s


In [4]:
#gives optimal parameters for Frequency model
tuner1.results_summary()

Results summary
Results in my_dir1/helloworld
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 1 summary
Hyperparameters:
units1: 64
activation1: tanh
units2: 80
activation2: sigmoid
units3: 88
activation3: relu
learning_rate: 0.01
Score: 0.09813291827837627

Trial 0 summary
Hyperparameters:
units1: 192
activation1: relu
units2: 224
activation2: relu
units3: 184
activation3: sigmoid
learning_rate: 0.01
Score: 0.09828082472085953

Trial 2 summary
Hyperparameters:
units1: 512
activation1: tanh
units2: 256
activation2: tanh
units3: 136
activation3: sigmoid
learning_rate: 0.001
Score: 0.09837942322095235

Trial 3 summary
Hyperparameters:
units1: 192
activation1: relu
units2: 160
activation2: tanh
units3: 56
activation3: sigmoid
learning_rate: 0.001
Score: 0.09854529798030853

Trial 4 summary
Hyperparameters:
units1: 352
activation1: relu
units2: 192
activation2: sigmoid
units3: 168
activation3: sigmoid
learning_rate: 0.01
Score: 0.09864064306020737


In [5]:
# Load necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
from kerastuner import Objective


# Define the model-building function
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Dense(units=hp.Int('units1', min_value=32, max_value=512, step=32),
                               input_shape=(9,),
                               activation=hp.Choice('activation1', values=['relu', 'tanh', 'sigmoid'])))  # Activation for first layer
        model.add(layers.Dense(units=hp.Int('units2', min_value=16, max_value=256, step=16),
                               activation=hp.Choice('activation2', values=['relu', 'tanh', 'sigmoid'])))
        model.add(layers.Dense(units=hp.Int('units3', min_value=8, max_value=128, step=16),
                               activation=hp.Choice('activation3', values=['relu', 'tanh', 'sigmoid'])))
        model.add(layers.Dense(units=hp.Int('units4', min_value=8, max_value=128, step=16),
                               activation=hp.Choice('activation4', values=['relu', 'tanh', 'sigmoid'])))
        model.add(layers.Dense(units=1, activation='linear'))

        model.compile(
            optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='mean_absolute_error'
        )
        return model

# Initialize the tuner
tuner2 = RandomSearch(
    MyHyperModel(),
    objective=Objective('val_loss', direction='min'),  # Use 'val_loss' since you're using mean squared error as loss
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir2',
    project_name='helloworld'
)




In [6]:
# Fit the tuner with batch size and epochs
tuner2.search(X_train, y_train_sev, epochs=50, validation_data=(X_test, y_test_sev))

Trial 5 Complete [00h 04m 06s]
val_loss: 1421.7837320963542

Best val_loss So Far: 1421.1111246744792
Total elapsed time: 00h 22m 59s


In [7]:
#gives optimal parameters for Severity model
tuner2.results_summary()

Results summary
Results in my_dir2/helloworld
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0 summary
Hyperparameters:
units1: 256
activation1: sigmoid
units2: 32
activation2: tanh
units3: 72
activation3: tanh
units4: 88
activation4: tanh
learning_rate: 0.01
Score: 1421.1111246744792

Trial 3 summary
Hyperparameters:
units1: 352
activation1: sigmoid
units2: 192
activation2: sigmoid
units3: 8
activation3: tanh
units4: 120
activation4: relu
learning_rate: 0.001
Score: 1421.1116536458333

Trial 1 summary
Hyperparameters:
units1: 416
activation1: sigmoid
units2: 48
activation2: sigmoid
units3: 8
activation3: relu
units4: 88
activation4: relu
learning_rate: 0.001
Score: 1421.1159261067708

Trial 4 summary
Hyperparameters:
units1: 32
activation1: relu
units2: 64
activation2: tanh
units3: 120
activation3: relu
units4: 40
activation4: tanh
learning_rate: 0.01
Score: 1421.7837320963542

Trial 2 summary
Hyperparameters:
units1: 256
activation1: relu
units2: 160
activa